# Decepticons: Corrupted Transformers Breach Privacy in Federated Learning for Language Models

This notebook shows an example for the threat model and attack described in "Decepticons: Corrupted Transformers Breach Privacy in Federated Learning for Language Models
". This example deviates from the other "honest-but-curious" server models and investigates a malicious server that may send malicious server updates. The attack succeeds for a range of common transformer architectures and works merely by sending a single malicious query to the user model.

In this notebook, we attack the (small) GPT-2 model (120mil parameters).



Paper URL: https://arxiv.org/abs/2201.12675

### Abstract:
A central tenet of Federated learning (FL), which trains models without centralizing user data, is privacy. However, previous work has shown that the gradient updates used in FL can leak user information. While the most industrial uses of FL are for text applications (e.g. keystroke prediction), nearly all attacks on FL privacy have focused on simple image classifiers. We propose a novel attack that reveals private user text by deploying malicious parameter vectors, and which succeeds even with mini-batches, multiple users, and long sequences. Unlike previous attacks on FL, the attack exploits characteristics of both the Transformer architecture and the token embedding, separately extracting tokens and positional embeddings to retrieve high-fidelity text. This work suggests that FL on text, which has historically been resistant to privacy attacks, is far more vulnerable than previously thought.

### Startup

In [1]:
try:
    import breaching
except ModuleNotFoundError:
    # You only really need this safety net if you want to run these notebooks directly in the examples directory
    # Don't worry about this if you installed the package or moved the notebook to the main directory.
    import os; os.chdir("..")
    import breaching
    
import torch
%load_ext autoreload
%autoreload 2

# Redirects logs directly into the jupyter notebook
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

### Initialize cfg object and system setup:

This will load the full configuration object. This includes the configuration for the use case and threat model as `cfg.case` and the hyperparameters and implementation of the attack as `cfg.attack`. All parameters can be modified below, or overriden with `overrides=` as if they were cmd-line arguments.

In [2]:
cfg = breaching.get_config(overrides=["attack=decepticon", "case=10_causal_lang_training", 
                                     "case/server=malicious-transformer"])
          
device = torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=getattr(torch, cfg.case.impl.dtype))
setup

Investigating use case causal_lang_training with server type malicious_transformer_parameters.


{'device': device(type='cpu'), 'dtype': torch.float32}

### Modify config options here

You can use `.attribute` access to modify any of these configurations for the attack, or the case:

In [3]:
cfg.case.user.num_data_points = 8 # How many sentences?
cfg.case.user.user_idx = 1 # From which user?
cfg.case.data.shape = [512] # This is the sequence length

cfg.case.server.provide_public_buffers = True # Send server signal to disable dropout
cfg.case.server.has_external_data = True  # Not strictly necessary, but could also use random text (see Appendix)
cfg.case.data.tokenizer = "gpt2"
cfg.case.model = "gpt2" # Could also choose "gpt2S" which contains ReLU activations
cfg.case.server.pretrained = False

## Attack hyperparameters:

# Server side:
cfg.case.server.param_modification.v_length = 32 # Length of the sentence component
cfg.case.server.param_modification.eps = 1e-16
cfg.case.server.param_modification.measurement_scale=1e12 # Circumvent GELU
cfg.case.server.param_modification.imprint_sentence_position = 0
cfg.case.server.param_modification.softmax_skew = 1e8
cfg.case.server.param_modification.sequence_token_weight = 1


# Attacker side:

# this option requires installation of `k-means-constrained` which can be tricky:
# If this doesn't work for you, falling back to "dynamic-threshold" is still a decent option.
cfg.attack.sentence_algorithm = "k-means" 
cfg.attack.token_strategy="embedding-norm" # no decoder bias in GPT
cfg.attack.embedding_token_weight=0.0 # Setting e.g. 0.25 here can improve performance slightly for long sequences

### Instantiate all parties

The following lines generate "server, "user" and "attacker" objects and print an overview of their configurations.

In [4]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Model architecture gpt2 loaded with 124,439,808 parameters and 12,582,924 buffers.
Overall this is a data ratio of   30381:1 for target shape [8, 512] given that num_queries=1.
User (of type UserSingleStep) with settings:
    Number of data points: 8

    Threat model:
    User provides labels: False
    User provides buffers: False
    User provides number of data points: True

    Data:
    Dataset: wikitext
    user: 1
    
        
Server (of type MaliciousTransformerServer) with settings:
    Threat model: Malicious (Parameters)
    Number of planned queries: 1
    Has external/public data: True

    Model:
        model specification: gpt2
        model state: default
        public buffers: True

    Sec

### Simulate an attacked FL protocol

This exchange is a simulation of a single query in a federated learning protocol. The server sends out a `server_payload` and the user computes an update based on their private local data. This user update is `shared_data` and contains, for example, the parameter gradient of the model in the simplest case. `true_user_data` is also returned by `.compute_local_updates`, but of course not forwarded to the server or attacker and only used for (our) analysis.

In [5]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)  

Found attention of shape torch.Size([2304, 768]).
Computing feature distribution before the probe layer Conv1D() from external data.
Feature mean is 6976836096.0, feature std is 694245261312.0.
Computing user update in model mode: eval.


In [6]:
user.print(true_user_data)

 The Tower Building of the Little Rock Arsenal, also known as U.S. Arsenal Building, is a building located in MacArthur Park in downtown Little Rock, Arkansas. Built in 1840, it was part of Little Rock's first military installation. Since its decommissioning, The Tower Building has housed two museums. It was home to the Arkansas Museum of Natural History and Antiquities from 1942 to 1997 and the MacArthur Museum of Arkansas Military History since 2001. It has also been the headquarters of the Little Rock Æsthetic Club since 1894. 
 The building receives its name from its distinct octagonal tower. Besides being the last remaining structure of the original Little Rock Arsenal and one of the oldest buildings in central Arkansas, it was also the birthplace of General Douglas MacArthur, who became the supreme commander of US forces in the South Pacific during World War II. It was also the starting place of the Camden Expedition. In 2011 it was named as one of the top 10 attractions in the s

### Reconstruct user data:

Now we launch the attack, reconstructing user data based on only the `server_payload` and the `shared_data`. 

For this attack, we also share secret information from the malicious server with the attack (`server.secrets`), which here is the location and structure of the imprint block.

In [7]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], server.secrets, 
                                                      dryrun=cfg.dryrun)

Proceeded to cut estimated token distribution at 1.50.
Recovered tokens tensor([   11,    11,    11,  ..., 50203, 50210, 50210]) through strategy embedding-norm.
Recovered 3878 embeddings with positional data from imprinted layer.
Assigned [482, 489, 489, 489, 471, 496, 478, 484] breached embeddings to each sentence.


Next we'll evaluate metrics, comparing the `reconstructed_user_data` to the `true_user_data`.

In [8]:
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, order_batch=True, compute_full_iip=False, 
                                    cfg_case=cfg.case, setup=setup)

METRICS: | Accuracy: 0.9224 | S-BLEU: 0.81 | FMSE: 1.8399e-03 | 
 G-BLEU: 0.78 | ROUGE1: 0.91| ROUGE2: 0.82 | ROUGE-L: 0.91| Token Acc: 96.09% | Label Acc: 96.09%


And finally, we also plot the reconstructed data:

In [9]:
user.print(reconstructed_user_data)

 any Tower Building of the Little Rock Arsenal, also known as U.S. Arsenal Building, morning a building located in MacArthur Park in downtown Little Rock, Arkansas. Built in 1840, situation was part of Little Rock 'ier first militaryals. Since its decommissioning, but Tower Building carrying housed two museums. It was home property the Arkansas Museum of Natural delivery and Antiquities from 1942 to 1997 and the MacArthur Museum of Arkansas Military History since 2001. It has also been the headquarters of the Little Rock Æsthetic Club since 1894.  state The building receives its name from its distinct octotten tower. Besides being the last remaining structure of the original Little Rock Arsenal and one of the oldest buildings in Travel Arkansas, it was also the birthplace of General Douglas MacArthur, who became the supreme commander of US forces in the South Pacific invited World War II. It 8 also the starting place of the Camden Expedition. In 2011 it was named as one owned the top 1

### Notes:
* There are a variety of hyperparameters to the attack which are set to reasonable defaults. Performance of the attack could be improved in some unusual use cases (datasets or models) by tuning these parameters further.
* In this example, dropout is disabled under the assumption that this is a parameter that can be controlled in the server update. The optimal attack simply disables dropout. However, the attack can still succeed when dropout is enforced by the user, albeit with a minor loss in reconstruction quality.
* This example also assumes complete freedom to choose the parameter vector, for this reason we circumvent the smooth part of the GELU activation with a "very" large measurement vector magnitude. This is arguably excessive for only a small again in accuracy.
* We also want to re-emphasize that the design space of these parameter modification attacks is large. A defense against the specific parameter modification described here is unlikely to be safe in general!